# Tutorial - MLFlow experiements monitoring

In this notebook, we will see how to smonitor your experiments using the integrated **mlflow** callbacks.

In [1]:
# Install the library
%pip install pythae

Note: you may need to restart the kernel to use updated packages.


## Train your Pythae model

In [2]:
import torchvision.datasets as datasets

%load_ext autoreload
%autoreload 2

/home/clement/anaconda3/envs/pythae_dev/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /home/clement/anaconda3/envs/pythae_dev/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [3]:
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=None)

train_dataset = mnist_trainset.data[:-10000].reshape(-1, 1, 28, 28) / 255.
eval_dataset = mnist_trainset.data[-10000:].reshape(-1, 1, 28, 28) / 255.

In [4]:
from pythae.models import BetaVAE, BetaVAEConfig
from pythae.trainers import BaseTrainerConfig
from pythae.pipelines.training import TrainingPipeline
from pythae.models.nn.benchmarks.mnist import Encoder_ResNet_VAE_MNIST, Decoder_ResNet_AE_MNIST

In [5]:
training_config = BaseTrainerConfig(
    output_dir='my_model',
    learning_rate=1e-4,
    batch_size=100,
    num_epochs=10, # Change this to train the model a bit more
)


model_config = BetaVAEConfig(
    input_dim=(1, 28, 28),
    latent_dim=16,
    beta=2.

)

model = BetaVAE(
    model_config=model_config,
    encoder=Encoder_ResNet_VAE_MNIST(model_config), 
    decoder=Decoder_ResNet_AE_MNIST(model_config) 
)

## Before lauching the pipeline, you will need to build your `MLFLowCallback`

To be able to access this feature you will need:
- the `mlflow` package installed in your virtual env. You can install it by running (`pip install mlflow`)

In [6]:
# Before being allowed to monitor your experiments you may need to run the following
#!pip install mlflow

In [7]:
# Create you callback
from pythae.trainers.training_callbacks import MLFlowCallback

callbacks = [] # the TrainingPipeline expects a list of callbacks

mlflow_cb = MLFlowCallback() # Build the callback 

# SetUp the callback 
mlflow_cb.setup(
    training_config=training_config, # training config
    model_config=model_config, # model config
    run_name="mlflow_cb_example", # specify your wandb project
)

callbacks.append(mlflow_cb) # Add it to the callbacks list

MLflow run started with run_id=8a40983e813249afa8a9f766ca5bc939


In [ ]:
pipeline = TrainingPipeline(
    training_config=training_config,
    model=model
)

In [ ]:
pipeline(
    train_data=train_dataset,
    eval_data=eval_dataset,
    callbacks=callbacks # pass the callbacks to the TrainingPipeline and you are done!
)

In [ ]:
import mlflow
mlflow.end_run()